In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-160988
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-160988


In [24]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
vm_size = "Standard_D2_V2"
max_nodes = 4

clusterName = "MyCluster"
# To use a different region for the compute, add a location='<region>' parameter
compute_config = AmlCompute.provisioning_configuration(
    vm_size=vm_size,
    max_nodes=max_nodes
)
cpu_cluster = ComputeTarget.create(ws, clusterName, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        'C':uniform(0.005, 20),
        'max_iter': uniform(20,2000),
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory='./',
    entry_script='train.py',
    compute_target=cpu_cluster
    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50,
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [26]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_a1a173e7-0c27-4773-8e53-c1cbeefa63b9
Web View: https://ml.azure.com/runs/HD_a1a173e7-0c27-4773-8e53-c1cbeefa63b9?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-160988/workspaces/quick-starts-ws-160988&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-10-14T00:03:18.694690][API][INFO]Experiment created<END>\n""<START>[2021-10-14T00:03:19.176031][GENERATOR][INFO]Trying to sample '50' jobs from the hyperparameter space<END>\n""<START>[2021-10-14T00:03:19.602796][GENERATOR][INFO]Successfully sampled '50' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_a1a173e7-0c27-4773-8e53-c1cbeefa63b9
Web View: https://ml.azure.com/runs/HD_a1a173e7-0c27-4773-8e53-c1cbeefa63b9?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-160988/workspaces/quick-starts-ws-160988&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "err

{'runId': 'HD_a1a173e7-0c27-4773-8e53-c1cbeefa63b9',
 'target': 'MyCluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-10-14T00:03:18.238087Z',
 'endTimeUtc': '2021-10-14T00:16:29.14278Z',
 'services': {},
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'propertie

In [9]:
import joblib
# Get your best run and save the model from that run.

if hyperdrive_run.status == 'Running':
    print("Still Running")
else:
    best = hyperdrive_run.get_best_run_by_primary_metric()
    
    if best is None:
        print("IT FAILED AGAIN")
    else:
        print(best.get_details()['runDefinition']['arguments'])
        print(best.get_metrics())

IT FAILED AGAIN


In [27]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
pth= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory().from_delimited_files(pth)

In [21]:
from train import clean_data

x, y = clean_data(ds)

In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [23]:
# Submit your automl run

myRun = exp.submit(automl_config, show_output=True)
myRun.wait_for_completion()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_52edd007-7f7d-4907-80e8-8dea3e27c67f,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

Cannot serialize JSON possibly due to timestamp, retrying


{'runId': 'AutoML_52edd007-7f7d-4907-80e8-8dea3e27c67f',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-10-13T23:39:08.938468Z',
 'endTimeUtc': '2021-10-14T00:00:21.291515Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-160988","workspace_name":"quick-starts-ws-160988","region":"southcentralus","compute_target":"local","spark_service":nul

In [29]:
# Retrieve and save your best automl model.

best, model = myRun.get_output()
print(best, model)

Run(Experiment: udacity-project,
Id: AutoML_52edd007-7f7d-4907-80e8-8dea3e27c67f_30,
Type: None,
Status: Completed) Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=1.7708333333333335, subsample=0.9, tree_method='auto'))], verbose=False)), ('7', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], v

In [30]:
joblib.dump(model,'automlModel.joblib')

['automlModel.joblib']

In [31]:
cpu_cluster.delete()